# galaxyEmulator step-by-step #

### Import the modules

In [1]:
import sys
import os
import numpy as np
from astropy.cosmology import Planck15
import h5py

In [2]:
sys.path.append('..')

In [3]:
from galaxyEmulator.utils import *
from galaxyEmulator.config import *
from galaxyEmulator.prepare import *
from galaxyEmulator.postprocess import *

### Get config and check if conflictions exist

In [4]:
# get default config
config = get_config()

In [5]:
config

{'filePath': '../../TNG-100',
 'workingDir': 'run',
 'simulationMode': 'ExtinctionOnly',
 'includeDust': False,
 'dustEmissionType': 'Equilibrium',
 'dustModel': 'ZubkoDustMix',
 'minWavelength': '0.1',
 'maxWavelength': '1.2',
 'boxLengthScale': '20',
 'maxBoxLength': '300',
 'wavelengthGrid': 'Linear',
 'numWavelengths': '300',
 'minLevel': '6',
 'maxLevel': '10',
 'numPackets': '1e7',
 'SEDFamily': 'BC03',
 'initialMassFunction': 'Chabrier',
 'minStellarMass': '1',
 'maxStellarMass': 'inf',
 'numViews': '3',
 'randomViews': False,
 'inclinations': '0,90,180',
 'azimuths': '0,90,180',
 'FoVboxLengthRatio': '1',
 'filters': 'CSST.NUV,CSST.u,CSST.g,CSST.r,CSST.i,CSST.z,CSST.y',
 'pixelScales': '0.074,0.074,0.074,0.074,0.074,0.074,0.074',
 'postProcessing': True,
 'numExposure': '4,2,2,2,2,2,4',
 'exposureTime_CSST': '150',
 'mirrorSize_CSST': '1',
 'resolFromPix': True,
 'resolution': '100',
 'includePSF': True,
 'PSFFromFile': True,
 'includeBackground': True,
 'backgroundSigma': '10.

In [6]:
# check if confictions exists in config
# flags > 0 means conflictions exist, 
# Please edit config according to the output message
flags = check_config(config)
print('issue flag: ', flags)

Configuration conflicts are indicated in RED
issue flag:  0


### Modify config and save it

In [7]:
config['filePath'] = '../../TNG-100'
config['filters'] = 'CSST.NUV,CSST.u,CSST.g,CSST.r,CSST.i,CSST.z,CSST.y'
config['includePSF'] = False
config['includeBackground'] = False
config['numGeneration'] = 5
config['saveDatacube'] = True
config['includeDust'] = True
config['resolFromPix'] = True
config['RGBFilters_CSST'] = 'g,r,z'

In [8]:
config

{'filePath': '../../TNG-100',
 'workingDir': 'run',
 'simulationMode': 'ExtinctionOnly',
 'includeDust': True,
 'dustEmissionType': 'Equilibrium',
 'dustModel': 'ZubkoDustMix',
 'minWavelength': '0.1',
 'maxWavelength': '1.2',
 'boxLengthScale': '20',
 'maxBoxLength': '300',
 'wavelengthGrid': 'Linear',
 'numWavelengths': '300',
 'minLevel': '6',
 'maxLevel': '10',
 'numPackets': '1e7',
 'SEDFamily': 'BC03',
 'initialMassFunction': 'Chabrier',
 'minStellarMass': '1',
 'maxStellarMass': 'inf',
 'numViews': '3',
 'randomViews': False,
 'inclinations': '0,90,180',
 'azimuths': '0,90,180',
 'FoVboxLengthRatio': '1',
 'filters': 'CSST.NUV,CSST.u,CSST.g,CSST.r,CSST.i,CSST.z,CSST.y',
 'pixelScales': '0.074,0.074,0.074,0.074,0.074,0.074,0.074',
 'postProcessing': True,
 'numExposure': '4,2,2,2,2,2,4',
 'exposureTime_CSST': '150',
 'mirrorSize_CSST': '1',
 'resolFromPix': True,
 'resolution': '100',
 'includePSF': False,
 'PSFFromFile': True,
 'includeBackground': False,
 'backgroundSigma': '10

In [9]:
flags = check_config(config)

Configuration conflicts are indicated in RED


In [10]:
# Now the flag == 0, we can proceed
flags

0

In [11]:
# saving is optional, as we can use the config in memory
# config = save_config(config)

### Load the subhalos and generate the .ski file

In [12]:
# get the snapshot num we want to use
snapnum = np.int32(config['snapNum'])

In [13]:
# get the redshift of current snapshot
snap = h5py.File(os.path.join(config['filePath'], 
                     f'snapdir_{snapnum:03d}/snap_{snapnum:03d}.0.hdf5'), 'r')
snapz = dict(snap['Header'].attrs.items())['Redshift']

In [14]:
# get the subhalos
subhalos = get_subhalos(config['filePath'], snapnum)

In [15]:
# get the min and max stellar mass for subhalos
minStellarMass = np.float32(config['minStellarMass'])
if config['maxStellarMass'] == 'inf':
    maxStellarMass = np.inf
else:
    maxStellarMass = np.float32(config['maxStellarMass'])

In [16]:
# get the SubhaloIDs following the stellar mass condition 
stellarMass = subhalos['SubhaloMassType'][:, 4] / Planck15.h
    
subhalo_indices = np.where((stellarMass > minStellarMass) \
                        & (stellarMass < maxStellarMass) \
                        & (subhalos['SubhaloFlag'] == 1) \
                        & (subhalos['SubhaloParent'] == 0))[0]

subhaloNums = subhalo_indices.shape[0]
print(f'Number of subhalos is {subhaloNums} in snapshot {snapnum} in stellar mass range {minStellarMass} to {maxStellarMass} [10^10 M_sun]')

Number of subhalos is 6041 in snapshot 94 in stellar mass range 1.0 to inf [10^10 M_sun]


In [17]:
subhalo_indices

array([     0,      1,      2, ..., 644966, 646870, 649674])

In [18]:
# Let us generate the galaxy from subhalo with ID = 1
subhaloID = 2

In [19]:
print(f'Stellar Mass: {stellarMass[subhaloID]} [10^10 M_sun]')

Stellar Mass: 19.245565835092847 [10^10 M_sun]


In [20]:
boxLength = subhalos['SubhaloHalfmassRadType'][:, 4][subhaloID] \
        * np.float32(config['boxLengthScale']) / Planck15.h

boxLength = np.min([boxLength, np.float32(config['maxBoxLength'])])
print('boxLength in kpc: ', np.around(boxLength, 2))

boxLength in kpc:  262.65


In [21]:
# create the working directory for SKIRT
workingDir = config['workingDir']
os.makedirs(workingDir, exist_ok=True)

In [22]:
# get and save the star-forming, quenched or dust particles
particles_from_tng(subhaloID, snapnum, snapz, subhalos, boxLength, config)

Star-forming star particles: 8
Quenched star particles: 202041
DUST particles: 0


In [23]:
# create .ski file for SKIRT
fixedRedshift = np.float32(config['fixedRedshift'])
properties = modify_ski_file(fixedRedshift, boxLength, config)

------estimate memory usage------
numViews: 3
numSpatialPixels: 2638
numWavelengthPixels: 300
Estimated memory usage: 50.105 GB


In [24]:
properties

{'redshift': np.float32(0.06),
 'lumiDis': np.float64(277.5121779933121),
 'resolution': [np.float64(99.56085841859017),
  np.float64(99.56085841859017),
  np.float64(99.56085841859017),
  np.float64(99.56085841859017),
  np.float64(99.56085841859017),
  np.float64(99.56085841859017),
  np.float64(99.56085841859017)],
 'angleRes': [0.074, 0.074, 0.074, 0.074, 0.074, 0.074, 0.074],
 'pixelscales_in_sr': [np.float64(1.287102616321e-13),
  np.float64(1.287102616321e-13),
  np.float64(1.287102616321e-13),
  np.float64(1.287102616321e-13),
  np.float64(1.287102616321e-13),
  np.float64(1.287102616321e-13),
  np.float64(1.287102616321e-13)],
 'baseRes': np.float64(99.56085841859017),
 'inclinations': [0.0, 90.0, 180.0],
 'azimuths': [0.0, 90.0, 180.0],
 'FoV': np.float64(262650.3875975698)}

### run SKIRT

In [26]:
# run SKIRT, this will take some time and memory
run_skirt(config)

23/09/2024 14:28:28.070   Welcome to SKIRT v9.0 (git 382ad4b built on 15/08/2024 at 01:46:45)
23/09/2024 14:28:28.070   Running on 2488h for xczhou
23/09/2024 14:28:28.070   Constructing a simulation from ski file 'skirt.ski'...
23/09/2024 14:28:28.126   Starting simulation skirt using 12 threads and a single process...
23/09/2024 14:28:28.126   Starting setup...
23/09/2024 14:28:28.126     Panchromatic wavelength regime
23/09/2024 14:28:28.126     With transfer medium
23/09/2024 14:28:28.126     Redshift: 0.06
23/09/2024 14:28:28.126     Luminosity distance: 277.531 Mpc
23/09/2024 14:28:28.126     Model and grid symmetry: 3D
23/09/2024 14:28:28.126     Photon life cycle: no explicit absorption; with forced scattering
23/09/2024 14:28:28.126   BruzualCharlotSEDFamily opened stored table /share/xczhou/SKIRT/resources/SKIRT9_Resources_Core/SED/BruzualCharlotSEDFamily_Chabrier_hr.stab
23/09/2024 14:28:28.126   ParticleSource reads smoothed source particles from text file /share/xczhou/ill

0

### postprocessing for calculating bandpass images, save them and clean the workingDir

In [27]:
postprocess(subhaloID, properties, config)

Considered survey(s) ['CSST']
